<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-djords1/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Sofia Djordjevic

**Student Number**: 400310764

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [33]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/Customer Dataset.csv')

In [34]:
if not df.empty:
    print("Original DataFrame head:")
    display(df.head())
    print("Original DataFrame info:")
    df.info()

    initial_rows = df.shape[0]
    df.dropna(inplace=True)
    print(f"\nRemoved missing values. Rows before: {initial_rows}, Rows after: {df.shape[0]}")

    # Remove any column of constant values
    cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
        print(f"Removed constant columns: {cols_to_drop}")
    else:
        print("No constant columns found.")

    # Convert the column Dt_Customer to number of days the customer has been with the company.
    # Using format='mixed' to handle potentially varied date formats, as suggested by the error.
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed')
    max_date = df['Dt_Customer'].max()
    df['Days_Since_Enrollment'] = (max_date - df['Dt_Customer']).dt.days
    df.drop(columns=['Dt_Customer'], inplace=True)
    print("\nConverted 'Dt_Customer' to 'Days_Since_Enrollment'.")

    print("\nPrepared DataFrame head:")
    display(df.head())
    print("Prepared DataFrame info:")
    df.info()

Original DataFrame head:


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


Original DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurch

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Days_Since_Enrollment
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,4,7,0,0,0,0,0,0,1,971
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,2,5,0,0,0,0,0,0,0,125
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,10,4,0,0,0,0,0,0,0,472
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,4,6,0,0,0,0,0,0,0,65
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,6,5,0,0,0,0,0,0,0,321


Prepared DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     2216 non-null   int64  
 1   Year_Birth             2216 non-null   int64  
 2   Education              2216 non-null   object 
 3   Marital_Status         2216 non-null   object 
 4   Income                 2216 non-null   float64
 5   Kidhome                2216 non-null   int64  
 6   Teenhome               2216 non-null   int64  
 7   Recency                2216 non-null   int64  
 8   MntWines               2216 non-null   int64  
 9   MntFruits              2216 non-null   int64  
 10  MntMeatProducts        2216 non-null   int64  
 11  MntFishProducts        2216 non-null   int64  
 12  MntSweetProducts       2216 non-null   int64  
 13  MntGoldProds           2216 non-null   int64  
 14  NumDealsPurchases      2216 non-null

# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [35]:
from sklearn.model_selection import train_test_split

# Define the target variable (y) and features (X)
y = df['MntWines']
X = df.drop(columns=['MntWines'])

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X: (2216, 26)
Shape of y: (2216,)

Shape of X_train: (1772, 26)
Shape of X_test: (444, 26)
Shape of y_train: (1772,)
Shape of y_test: (444,)


(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Identify categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns
numerical_features = X_train.select_dtypes(exclude=['object']).columns

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Keep numerical features as is
)

# Define the Random Forest Regressor model with specified hyperparameters
# Using some default values for illustration, will be tuned later
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=20 # Ensure reproducibility
)

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', rf_model)])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

print("Machine learning pipeline built and fitted successfully!")

Machine learning pipeline built and fitted successfully!


(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [37]:
import pandas as pd

# Create a DataFrame for the new customer's features

# Recalculate max_date from the original Dt_Customer column from the raw CSV for consistency.
# This ensures the 'Days_Since_Enrollment' is calculated relative to the same 'max_date' as the training data.
original_df = pd.read_csv('/content/Customer Dataset.csv')
original_df['Dt_Customer'] = pd.to_datetime(original_df['Dt_Customer'], format='mixed')
max_date_original_df = original_df['Dt_Customer'].max()

new_customer_dt_str = '2016-10-10'
new_customer_dt = pd.to_datetime(new_customer_dt_str)
new_customer_days_since_enrollment = (max_date_original_df - new_customer_dt).days

new_customer_data = {
    'ID': 9999, # Dummy ID
    'Year_Birth': 2021 - 48, # Approximate Year_Birth from Age 48 (assuming current year is 2021 as a reference)
    'Education': 'Graduation',
    'Marital_Status': 'Married',
    'Income': 80000.0,
    'Kidhome': 1,
    'Teenhome': 1,
    'Recency': 43,
    'MntFruits': 0, # Added missing column with assumed value 0
    'MntMeatProducts': 0, # Added missing column with assumed value 0
    'MntFishProducts': 0, # Added missing column with assumed value 0
    'MntSweetProducts': 0, # Added missing column with assumed value 0
    'MntGoldProds': 0, # Added missing column with assumed value 0
    'NumDealsPurchases': 2,
    'NumWebPurchases': 1,
    'NumCatalogPurchases': 0,
    'NumStorePurchases': 15,
    'NumWebVisitsMonth': 5,
    'AcceptedCmp1': 0,
    'AcceptedCmp2': 0,
    'AcceptedCmp3': 0,
    'AcceptedCmp4': 0,
    'AcceptedCmp5': 0,
    'Complain': 0,
    'Response': 0, # Assuming 0 for a new customer prediction unless specified, as 'Response' is a feature in X_train
    'Days_Since_Enrollment': new_customer_days_since_enrollment
}

new_customer_df = pd.DataFrame([new_customer_data])

# Predict the wine spending using the fitted pipeline
predicted_spending = pipeline.predict(new_customer_df)

print(f"Predicted spending on wines for the new customer: ${predicted_spending[0]:.2f}")

Predicted spending on wines for the new customer: $401.57


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [38]:
from sklearn.metrics import r2_score, mean_absolute_error

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R2) on the test set: {r2:.2f}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE) on the test set: {mae:.2f}")

# Interpretation of the results
print("\n--- Model Performance Interpretation ---")
print(f"The R-squared score of {r2:.2f} indicates that approximately {r2*100:.0f}% of the variance in wine spending can be explained by our model's features. This suggests a reasonably good fit, though there's still a significant portion of variance not captured.")
print(f"The Mean Absolute Error (MAE) of {mae:.2f} means that, on average, our model's predictions for wine spending are off by approximately ${mae:.2f}. This gives a direct measure of the average magnitude of the errors in dollars.")
print("Overall, the model provides a fair prediction of wine spending, but there is room for improvement given the MAE value relative to the scale of wine spending. Further tuning or different models might enhance performance.")

R-squared (R2) on the test set: 0.78
Mean Absolute Error (MAE) on the test set: 88.66

--- Model Performance Interpretation ---
The R-squared score of 0.78 indicates that approximately 78% of the variance in wine spending can be explained by our model's features. This suggests a reasonably good fit, though there's still a significant portion of variance not captured.
The Mean Absolute Error (MAE) of 88.66 means that, on average, our model's predictions for wine spending are off by approximately $88.66. This gives a direct measure of the average magnitude of the errors in dollars.
Overall, the model provides a fair prediction of wine spending, but there is room for improvement given the MAE value relative to the scale of wine spending. Further tuning or different models might enhance performance.


(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [39]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Perform 6-fold cross-validation
# Using 'neg_mean_absolute_error' as the scoring metric, as it's common for regression and we want to maximize it.
# The result will be negative MAE, so we'll take the absolute value for interpretation.
cv_scores = cross_val_score(pipeline, X, y, cv=6, scoring='neg_mean_absolute_error', n_jobs=-1)

# Convert negative scores to positive MAE scores
mae_scores = -cv_scores

print(f"Cross-validation MAE scores (6 folds): {mae_scores}")
print(f"Mean CV MAE: {np.mean(mae_scores):.2f}")
print(f"Standard deviation of CV MAE: {np.std(mae_scores):.2f}")

print("\n--- Cross-Validation Interpretation ---")
print("The cross-validation results provide a more robust estimate of the model's performance by training and testing on different subsets of the data. The mean MAE across 6 folds is approximately "
      f"${np.mean(mae_scores):.2f} with a standard deviation of ${np.std(mae_scores):.2f}. This indicates that the model's performance is relatively consistent across different data splits, "
      "and the average error is similar to what was observed on the single test set in part (iv).")

Cross-validation MAE scores (6 folds): [75.71806575 87.55809692 91.11025649 79.48061188 83.69738606 82.05808584]
Mean CV MAE: 83.27
Standard deviation of CV MAE: 5.05

--- Cross-Validation Interpretation ---
The cross-validation results provide a more robust estimate of the model's performance by training and testing on different subsets of the data. The mean MAE across 6 folds is approximately $83.27 with a standard deviation of $5.05. This indicates that the model's performance is relatively consistent across different data splits, and the average error is similar to what was observed on the single test set in part (iv).


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [40]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, 10, 15],
    'regressor__min_samples_split': [3, 6],
    'regressor__min_samples_leaf': [2, 4, 8]
}

# Initialize GridSearchCV
# Using neg_mean_absolute_error as the scoring metric
grid_search = GridSearchCV(pipeline, param_grid, cv=6, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Display the best hyperparameter combination and its performance score
print("\n--- Hyperparameter Tuning Results ---")
print(f"Best Hyperparameter Combination: {grid_search.best_params_}")
print(f"Corresponding Best MAE (negative score, so positive value is actual MAE): {-grid_search.best_score_:.2f}")

# Retrieve the best model
best_rf_model = grid_search.best_estimator_
print("Best model retrieved successfully!")

Fitting 6 folds for each of 36 candidates, totalling 216 fits

--- Hyperparameter Tuning Results ---
Best Hyperparameter Combination: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 3, 'regressor__n_estimators': 100}
Corresponding Best MAE (negative score, so positive value is actual MAE): 83.92
Best model retrieved successfully!


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Create a pipeline for Linear Regression, reusing the preprocessor
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# Fit the Linear Regression pipeline to the training data
lr_pipeline.fit(X_train, y_train)

# Predict on the test data using the Linear Regression model
y_pred_lr = lr_pipeline.predict(X_test)

# Evaluate Linear Regression model performance
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print("--- Linear Regression Model Performance ---")
print(f"R-squared (R2) on the test set (Linear Regression): {r2_lr:.2f}")
print(f"Mean Absolute Error (MAE) on the test set (Linear Regression): {mae_lr:.2f}")

# --- Comparison with Best Random Forest Model ---
print("\n--- Comparison with Best Random Forest Model (from Q1) ---")

# Re-predict using the best_rf_model from GridSearchCV on the test set
y_pred_rf_best = best_rf_model.predict(X_test)

r2_rf_best = r2_score(y_test, y_pred_rf_best)
mae_rf_best = mean_absolute_error(y_test, y_pred_rf_best)

print(f"R-squared (R2) on the test set (Best Random Forest): {r2_rf_best:.2f}")
print(f"Mean Absolute Error (MAE) on the test set (Best Random Forest): {mae_rf_best:.2f}")

print("\n--- Performance Comparison Interpretation ---")
if r2_rf_best > r2_lr:
    print(f"The Best Random Forest model (R2: {r2_rf_best:.2f}) outperforms the Linear Regression model (R2: {r2_lr:.2f}) in terms of explained variance.")
else:
    print(f"The Linear Regression model (R2: {r2_lr:.2f}) performs similarly to or better than the Best Random Forest model (R2: {r2_rf_best:.2f}) in terms of explained variance.")

if mae_rf_best < mae_lr:
    print(f"The Best Random Forest model (MAE: {mae_rf_best:.2f}) has a lower average prediction error than the Linear Regression model (MAE: {mae_lr:.2f}).")
else:
    print(f"The Linear Regression model (MAE: {mae_lr:.2f}) has a lower or similar average prediction error compared to the Best Random Forest model (MAE: {mae_rf_best:.2f}).")
print("Overall, comparing the R2 and MAE values, we can determine which model provides a better balance of explained variance and prediction accuracy on unseen data.")

--- Linear Regression Model Performance ---
R-squared (R2) on the test set (Linear Regression): 0.67
Mean Absolute Error (MAE) on the test set (Linear Regression): 119.71

--- Comparison with Best Random Forest Model (from Q1) ---
R-squared (R2) on the test set (Best Random Forest): 0.79
Mean Absolute Error (MAE) on the test set (Best Random Forest): 87.51

--- Performance Comparison Interpretation ---
The Best Random Forest model (R2: 0.79) outperforms the Linear Regression model (R2: 0.67) in terms of explained variance.
The Best Random Forest model (MAE: 87.51) has a lower average prediction error than the Linear Regression model (MAE: 119.71).
Overall, comparing the R2 and MAE values, we can determine which model provides a better balance of explained variance and prediction accuracy on unseen data.


(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [42]:
import plotly.express as px
import pandas as pd
import numpy as np

# Step 1: Compute the absolute residual for each record in the test dataset
# Absolute residuals for Linear Regression
abs_residuals_lr = np.abs(y_test - y_pred_lr)

# Absolute residuals for Best Random Forest model
abs_residuals_rf = np.abs(y_test - y_pred_rf_best)

# Step 2: Create a DataFrame for the scatterplot
residuals_df = pd.DataFrame({
    'LR_Residuals': abs_residuals_lr,
    'RF_Residuals': abs_residuals_rf
})

# Step 3: Generate the scatterplot using plotly.express
fig = px.scatter(residuals_df, x='LR_Residuals', y='RF_Residuals',
                 title='Comparison of Absolute Residuals (LR vs. RF)',
                 labels={'LR_Residuals': 'Linear Regression Absolute Residuals',
                         'RF_Residuals': 'Random Forest Absolute Residuals'})

# Add a 45-degree reference line
# Determine min and max values for the reference line based on the data
min_val = min(residuals_df['LR_Residuals'].min(), residuals_df['RF_Residuals'].min())
max_val = max(residuals_df['LR_Residuals'].max(), residuals_df['RF_Residuals'].max())

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)

fig.show()

print("\n--- Interpretation of the Q-Q plot ---")
print("This plot serves as a visual comparison of the error distributions of the two models. ")
print("Points below the 45-degree (red dashed) line indicate instances where the Random Forest model's absolute residual is smaller than the Linear Regression model's, meaning RF performed better for those specific predictions.")
print("Conversely, points above the line suggest that Linear Regression had a smaller absolute residual for those predictions.")
print("A concentration of points below the line, especially further away from the origin, would suggest that the Random Forest model generally makes smaller errors than the Linear Regression model.")
print("The further a point is from the 45-degree line, the larger the difference in performance between the two models for that specific data point.")



--- Interpretation of the Q-Q plot ---
This plot serves as a visual comparison of the error distributions of the two models. 
Points below the 45-degree (red dashed) line indicate instances where the Random Forest model's absolute residual is smaller than the Linear Regression model's, meaning RF performed better for those specific predictions.
Conversely, points above the line suggest that Linear Regression had a smaller absolute residual for those predictions.
A concentration of points below the line, especially further away from the origin, would suggest that the Random Forest model generally makes smaller errors than the Linear Regression model.
The further a point is from the 45-degree line, the larger the difference in performance between the two models for that specific data point.


# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [43]:
# Calculate the 33.33% and 66.67% percentiles for MntMeatProducts
meat_spending_quantiles = df['MntMeatProducts'].quantile([1/3, 2/3])

lower_bound = meat_spending_quantiles[1/3]
upper_bound = meat_spending_quantiles[2/3]

print(f"33.33% percentile for MntMeatProducts: {lower_bound:.2f}")
print(f"66.67% percentile for MntMeatProducts: {upper_bound:.2f}")

# Define a function to label the spending
def label_meat_spending(spending):
    if spending < lower_bound:
        return 'low'
    elif spending > upper_bound:
        return 'high'
    else:
        return 'medium'

# Apply the function to create the new 'Meat_Spending_Category' column
df['Meat_Spending_Category'] = df['MntMeatProducts'].apply(label_meat_spending)

print("\nMeat Spending Category distribution:")
print(df['Meat_Spending_Category'].value_counts())

print("\nDataFrame head with new category:")
display(df[['MntMeatProducts', 'Meat_Spending_Category']].head())

33.33% percentile for MntMeatProducts: 24.00
66.67% percentile for MntMeatProducts: 146.33

Meat Spending Category distribution:
Meat_Spending_Category
medium    746
high      739
low       731
Name: count, dtype: int64

DataFrame head with new category:


,MntMeatProducts,Meat_Spending_Category
0,546,high
1,6,low
2,127,medium
3,20,low
4,118,medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [44]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Define the new target variable (y_meat) and features (X_meat)
y_meat = df['Meat_Spending_Category']
X_meat = df.drop(columns=['MntMeatProducts', 'Meat_Spending_Category', 'MntWines'])
# We drop MntWines to ensure X_meat is independent of wine spending (Q1's target) and avoid data leakage from MntMeatProducts

print("Shape of X_meat:", X_meat.shape)
print("Shape of y_meat:", y_meat.shape)

# Split the data into training and testing sets for meat spending prediction
X_meat_train, X_meat_test, y_meat_train, y_meat_test = train_test_split(X_meat, y_meat, test_size=0.2, random_state=20, stratify=y_meat)

print("\nShape of X_meat_train:", X_meat_train.shape)
print("Shape of X_meat_test:", X_meat_test.shape)
print("Shape of y_meat_train:", y_meat_train.shape)
print("Shape of y_meat_test:", y_meat_test.shape)

# Identify categorical and numerical features for the meat spending prediction task
categorical_features_meat = X_meat_train.select_dtypes(include=['object']).columns
numerical_features_meat = X_meat_train.select_dtypes(exclude=['object']).columns

# Create a column transformer for preprocessing for the KNN model
preprocessor_meat = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_meat),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_meat)
    ],
    remainder='passthrough' # Keep other columns as is (if any, though should be handled by num/cat)
)

# Define the K-nearest-neighbors classifier model
knn_model = KNeighborsClassifier(n_neighbors=5) # Using 5 neighbors as a common starting point

# Create the pipeline for KNN
pipeline_knn = Pipeline(steps=[('preprocessor', preprocessor_meat),
                               ('classifier', knn_model)])

# Fit the pipeline to the training data
pipeline_knn.fit(X_meat_train, y_meat_train)

print("\nKNN classification pipeline built and fitted successfully!")

Shape of X_meat: (2216, 25)
Shape of y_meat: (2216,)

Shape of X_meat_train: (1772, 25)
Shape of X_meat_test: (444, 25)
Shape of y_meat_train: (1772,)
Shape of y_meat_test: (444,)

KNN classification pipeline built and fitted successfully!


(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [45]:
from sklearn.metrics import classification_report

# Predict on the test data using the KNN pipeline
y_meat_pred = pipeline_knn.predict(X_meat_test)

# Generate the classification report
report = classification_report(y_meat_test, y_meat_pred)
print("Classification Report for Meat Spending Category:\n")
print(report)

print("\n--- Interpretation of the Classification Report ---")
print("The classification report provides key metrics for evaluating the KNN model's performance for each class ('high', 'low', 'medium').\n")

print("**Precision**: For each class, precision measures the proportion of correctly predicted positive observations among all predicted positive observations. For example, a high precision for 'high' spending means that when the model predicts 'high' spending, it is usually correct.\n")

print("**Recall**: For each class, recall measures the proportion of correctly predicted positive observations among all actual positive observations. For instance, a high recall for 'high' spending indicates that the model identifies most of the actual 'high' spending customers.\n")

print("**F1-score**: The F1-score is the harmonic mean of precision and recall. It gives a better measure of the incorrectly classified cases than accuracy, especially for imbalanced datasets. A high F1-score suggests that the model has good precision and recall for that class.\n")

print("**Support**: This is the number of actual occurrences of each class in the `y_meat_test` dataset. It tells us how many instances of each category ('high', 'low', 'medium') were in the test set.\n")

print("**Accuracy**: Overall, accuracy is the proportion of correctly predicted instances out of the total instances. It indicates how often the model is correct across all classes.\n")

print("**Macro Avg**: This is the average of the metric (precision, recall, f1-score) calculated independently for each class. It treats all classes equally, regardless of their support.\n")

print("**Weighted Avg**: This is the average of the metric weighted by the support (number of true instances) for each class. It accounts for class imbalance.\n")

print("By examining these metrics, particularly for each category ('low', 'medium', 'high'), we can understand how well our KNN model distinguishes between different levels of meat spending.")


Classification Report for Meat Spending Category:

              precision    recall  f1-score   support

        high       0.85      0.81      0.83       148
         low       0.76      0.88      0.82       146
      medium       0.67      0.59      0.63       150

    accuracy                           0.76       444
   macro avg       0.76      0.76      0.76       444
weighted avg       0.76      0.76      0.76       444


--- Interpretation of the Classification Report ---
The classification report provides key metrics for evaluating the KNN model's performance for each class ('high', 'low', 'medium').

**Precision**: For each class, precision measures the proportion of correctly predicted positive observations among all predicted positive observations. For example, a high precision for 'high' spending means that when the model predicts 'high' spending, it is usually correct.

**Recall**: For each class, recall measures the proportion of correctly predicted positive observations

# Describe how you used Gen. AI. in this assignment (2 points)

I used Gen. AI in this assignment to assist me with errors that I was receiving with formatting issues, and to help me describe the models' performances based on my results. Also, it helped me visualize the Q-Q plot to serve as a comparison of the error distributions of the two models.

**Note**: The remaining 5 points will be assigned to readability of the work.